In [541]:
import pymongo
from datetime import datetime
conn = pymongo.MongoClient('localhost',27017)

In [543]:
conn.drop_database('cmm')

## create the db

In [544]:
conn.list_database_names()

['admin', 'ce-store', 'config', 'demo', 'hertz', 'local']

In [73]:
cmm = conn.cmm

In [546]:
cluster = cmm.cluster
instance =  cmm.instance

In [51]:
cmm.list_collection_names()

['cluster']

In [538]:
cluster_info = {	
	"templateformat":"2010-09-09",
    "type": "nsdadd",
    "clustername": "MyCluster",
    "created_date": datetime.now() ,
    "properties": {
        "ClusterSettings": [
            {
                "Name": "containerInsights",
                "Value": "enabled"
            }
        ],
        "Tags": [
            {
                "Key": "environment",
                "Value": "production"
            },
                        {
                "Key": "storage",
                "Value": "persist"
            }
        ]
    }

}

In [539]:
cluster.find({"created_date":{"$exists":True}}).count()

/home/boombox/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


1

In [540]:
cluster.create_index("clustername", unique=True)
_data = cmm['cluster'].insert_one(cluster_info)

DuplicateKeyError: E11000 duplicate key error collection: cmm.cluster index: clustername_1 dup key: { : "MyCluster" }

In [525]:
pipeline = []

from pymongo import MongoClient, DESCENDING, ASCENDING
pipeline.extend([
    {
    "$match":
              {
                     "properties.Tags": { "$all": [
                                    { "$elemMatch" : { "Key": "environment",'Value': 'production'} },
                                    { "$elemMatch" : { "Key": "storage",'Value': 'persist'} }
                                  ] },
                   }
              
    }, 
    {
    "$sort": {"created_date": DESCENDING}
    },
    {
        "$project":{"_id":1}
    }
])

In [526]:
counting = pipeline[:]
count_stage = {"$count": "count"}
counting.append(count_stage)

In [527]:
movies_per_page = 2
page = 0

In [528]:
skip_stage = {"$skip": movies_per_page * page}
limit_stage = {"$limit": movies_per_page}

In [529]:
pipeline.append(skip_stage)
pipeline.append(limit_stage)

In [530]:
list(cluster.aggregate(pipeline, allowDiskUse=True))

[{'_id': ObjectId('5e3ee7618f6cf7093a2564d2')}]

In [531]:
movies = list(cluster.aggregate(pipeline, allowDiskUse=True))
count = list(cluster.aggregate(counting, allowDiskUse=True))[
    0].get("count")

In [532]:
movies,count

([{'_id': ObjectId('5e3ee7618f6cf7093a2564d2')}], 1)

In [515]:
movies

[{'_id': ObjectId('5e3ee70a8f6cf7093a2564d1')}]

In [502]:
abcd = cluster.delete_many(              {
                     "properties.Tags": { "$all": [
                                    { "$elemMatch" : { "Key": "environment",'Value': 'production'} },
                                    { "$elemMatch" : { "Key": "storage",'Value': 'persist'} }
                                  ] },
                   })

In [533]:
abcd = cluster.delete_one({"_id": movies[0]["_id"]})

1

In [545]:
pipeline = []

from pymongo import MongoClient, DESCENDING, ASCENDING
pipeline.extend([{'$match': {'instance_name': 'instance5', 'cluster_id': '5e4014d32c2714016b4463ff'}}, {'$sort': {'created_date': -1}}])

In [547]:
list(instance.aggregate(pipeline, allowDiskUse=True))

[{'_id': ObjectId('5e4014e52c2714016b446405'),
  'cluster_id': '5e4014d32c2714016b4463ff',
  'region': 'us-east-1a',
  'instance_name': 'instance5',
  'type': 'AWS::EC2::Instance',
  'properties': {'image': {'os': 'ubuntu',
    'virtualization': 'HVM',
    'Architecture': '32',
    'Storage': 'EBS'},
   'keypair': {'name': 'mycluster_key',
    'Format': 'SSH public key file format'},
   'instance_type': 'm1.small',
   'security_groups': {'name': 'some_name', 'VPC': 'some_id'},
   'tags': [{'key': 'environment', 'value': 'production'},
    {'key': 'storage', 'value': 'production'}],
   'ip_address': '62.33.41.248'},
  'created_date': datetime.datetime(2020, 2, 9, 19, 49, 17, 588000),
  'updated_date': datetime.datetime(2020, 2, 9, 19, 49, 17, 588000)}]

In [567]:
from bson import ObjectId
cluster_id = "5e4037627264ca4db7c4ee99"

In [568]:
abcd = instance.delete_many({"cluster_id":ObjectId(cluster_id)})

In [583]:
abc = instance.find({'properties.tags': {'$all': [{'$elemMatch': {'key': 'owner', 'value': 'TeamA'}}]}})

In [584]:
list(abc)[0]

{'_id': ObjectId('5e406a76c2253fbbd3094785'),
 'cluster_id': ObjectId('5e406a5fc2253fbbd3094783'),
 'region': 'us-east-1a',
 'instance_name': 'instance1',
 'type': 'AWS::EC2::Instance',
 'state': 'ON',
 'properties': {'image': {'os': 'ubuntu',
   'virtualization': 'HVM',
   'Architecture': '32',
   'Storage': 'EBS'},
  'keypair': {'name': 'mycluster_key', 'Format': 'SSH public key file format'},
  'instance_type': 'm1.small',
  'security_groups': {'name': 'some_name', 'VPC': 'some_id'},
  'tags': [{'key': 'owner', 'value': 'TeamA'},
   {'key': 'environment', 'value': 'production'}],
  'ip_address': '85.35.160.54'},
 'created_date': datetime.datetime(2020, 2, 10, 1, 54, 22, 612000),
 'updated_date': datetime.datetime(2020, 2, 10, 1, 54, 22, 612000)}

In [578]:
abc = instance.update_many({'properties.tags': {'$all': [{'$elemMatch': {'key': 'owner', 'value': 'TeamA'}}]}},
                          {'$set': {'state': 'ON'}}
                          )

TypeError: array_filters must be a list

In [575]:
abc.modified_count

3

In [11]:
pipeline = []
count = 0
sort = {"created_date": "DESCENDING"}
if "order" == 'asc':
    sort["created_date"] = "ASCENDING"

# match aggregation pipe
pipeline.extend([
    {
        "$match": {}
    }])
if True:
    pipeline.append({"$sort":sort})

In [12]:
pipeline

[{'$match': {}}, {'$sort': {'created_date': 'DESCENDING'}}]

In [8]:
pipeline

[{'$match': {}}, {'$sort': {'created_date': 'DESCENDING'}}]